In [1]:
import re, string
import pandas as pd
from unidecode  import unidecode
from cleanco import cleanco
from spellchecker import SpellChecker
import pyodbc

In [2]:
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=192.168.1.47; uid=seair; pwd=se@ir1234; Database = tempEXIM; Trusted_Connection = No')

In [3]:
df = pd.read_sql("""select *,'Turmeric'  from Special2020..Exporttable where Product like '%Turmeric%' and HSCode like '09%' union
select *,'Black Paper' from Special2020..Exporttable where Product like '%Black%' and Product like '%Paper%' and HSCode like '09%'""",conn)

In [11]:
def cleansing_special_characters(txt):
    seps = [' ',';',':','.',',','*','#','@','|','\\','-','_','?','%','!','^','(',')','$','=','+','"','<','>',"'",' AND ', ' and ']
    default_sep = seps[0]
    txt = str(txt)
    for sep in seps[1:]:
        if sep == " AND " or sep == " and ":
            txt = txt.upper()
            txt = txt.replace(sep, ' & ')
        else:
            txt = txt.upper()
            txt = txt.replace(sep, default_sep)
    try :
        list(map(int,txt.split()))
        txt = 'NUMBERS'
    except:
        pass
    txt = re.sub(' +', ' ', txt)
    temp_list = [i.strip() for i in txt.split(default_sep)]
    temp_list = [i for i in temp_list if i]
    return " ".join(temp_list)


# remove suffix
# terms = prepare_terms()
def  remove_suffix(txt):
    try:
        comp_temp = cleanco(txt)
        x_comp = comp_temp.clean_name()
        comp = cleanco(x_comp)
    except:
        return txt
#     x = cleanco(txt)
#     y = x.clean_name()
#     t = x.type()
#     z = cleanco(y)
#     final = z.clean_name()
    return comp.clean_name()


#########################################################
def find_suffix(x,y):
    x = str(x)
    y = str(y)
    x = re.sub(' +', ' ', x)
    y = re.sub(' +', ' ', y)
    x = x.split(' ')
    y = y.split(' ')
    not_in_y = [i for i in x if i not in y]
    return ' '.join(not_in_y)


####################################################################################

spell = SpellChecker()
def spell_correct(txt):
    return spell.correction(txt)



###########################################################

punctuation = re.compile('[%s]' % re.escape(string.punctuation))
class fingerprinter(object):
    
    # __init__function
    
    def __init__(self, string):
        self.string = self._preprocess(string)
        
    
    # strip leading, trailing spaces and to lower case
    def _preprocess(self, string):
        return punctuation.sub('',string.strip().lower())
    
        
    def _latinize(self, string):
        return unidecode(string)
#         return unidecode(string.decode('utf-8'))
    
    def _unique_preserve_order(self,seq):
        seen = set()
        seen_add = seen.add
        return [x for x in seq if not (x in seen or seen_add(x))]

    
    #-####################################################
    def get_fingerprint(self):
        return self._latinize(' '.join(self._unique_preserve_order(sorted(self.string.split()))))
    
    ######################################################
    def get_ngram_fingerprint(self, n=1):
        return self._latinize(''.join(self._unique_preserve_order(sorted([self.string[i:i + n] for i in range(len(self.string) - n +1)]))))
    
    

In [7]:
# df = pd.read_excel('Fingerprint test.xlsx')
# step 1
# df['clean'] = df['ForeignCompany'].apply(cleansing_special_characters)
#df['no_suffix'] = df['clean'].apply(remove_suffix)
# step 4
# df['noise_suffix'] = df[['clean','no_suffix']].apply(lambda x :find_suffix(x.clean, x.no_suffix), axis =1)
# df['fingerprint'] = df['no_suffix'].apply(lambda x : fingerprinter(x).get_fingerprint())
# df['n_gram_fingerprint_n=1'] = df['no_suffix'].apply(lambda x : fingerprinter(x.replace(' ','')).get_ngram_fingerprint())
# df['n_gram_fingerprint_n=2'] = df['no_suffix'].apply(lambda x : fingerprinter(x.replace(' ','')).get_ngram_fingerprint(n=2))
# df['n_gram_fingerprint_n=3'] = df['no_suffix'].apply(lambda x : fingerprinter(x.replace(' ','')).get_ngram_fingerprint(n=3))
# df.to_excel('fingerprinter_output.xlsx', index = False)
# df2 = df.drop_duplicates(subset=['fingerprint','n_gram_fingerprint_n=1'])
# df2.to_excel('fingerprint_rem_duplicates.xlsx', index = False)

In [8]:
# df = pd.read_excel('Fingerprint test.xlsx')
# df['fingerprint'] = df['Supp_Name'].apply(lambda x : fingerprinter(x).get_fingerprint())
# df['n_gram_fingerprint_n=2'] = df['Supp_Name'].apply(lambda x : fingerprinter(x.replace(' ','')).get_ngram_fingerprint(n=2))

In [9]:
# # step 2
# df['temp_suffix'] = df['clean'].apply(remove_suffix)
# df[['no_suffix','suffix']] = df['temp_suffix'].str.split('|',expand=True)

In [13]:
# #step 3
# df['Spell_check_no_suffix'] = df['no_suffix'].apply(spell_correct)

In [4]:
df = pd.read_excel('Unique_Supp_Name-2020.xlsx')

In [5]:
df.head()

,Uniq_Index,index,Supp_Name,SOUNDEX_CODE,M/S,Supp_Name1
0,0,0,YIWU GENG WEI COMMODITY PURCHASE CO,Y000,False,YIWU GENG WEI COMMODITY PURCHASE CO
1,1,4,SEW EURODRIVE GMBH & CO KG,S000,False,SEW EURODRIVE GMBH & CO KG
2,2,49,PAM TRADING CORP,P500,False,PAM TRADING CORP
3,3,54,MORGAN AM&T SHANGHAI CO LTD,M625,False,MORGAN AM&T SHANGHAI CO LTD
4,4,55,GROZ BECKERT KG,G620,False,GROZ BECKERT KG


In [12]:
df['No_Suffix'] = df['Supp_Name1'].apply(remove_suffix)

In [13]:
df.head()

,Uniq_Index,index,Supp_Name,SOUNDEX_CODE,M/S,Supp_Name1,No_Suffix
0,0,0,YIWU GENG WEI COMMODITY PURCHASE CO,Y000,False,YIWU GENG WEI COMMODITY PURCHASE CO,YIWU GENG WEI COMMODITY PURCHASE
1,1,4,SEW EURODRIVE GMBH & CO KG,S000,False,SEW EURODRIVE GMBH & CO KG,SEW EURODRIVE
2,2,49,PAM TRADING CORP,P500,False,PAM TRADING CORP,PAM TRADING
3,3,54,MORGAN AM&T SHANGHAI CO LTD,M625,False,MORGAN AM&T SHANGHAI CO LTD,MORGAN AM&T SHANGHAI
4,4,55,GROZ BECKERT KG,G620,False,GROZ BECKERT KG,GROZ BECKERT


In [14]:
df['suffix'] = df[['Supp_Name1','No_Suffix']].apply(lambda x :find_suffix(x.Supp_Name1, x.No_Suffix), axis =1)

In [15]:
df.head()

,Uniq_Index,index,Supp_Name,SOUNDEX_CODE,M/S,Supp_Name1,No_Suffix,suffix
0,0,0,YIWU GENG WEI COMMODITY PURCHASE CO,Y000,False,YIWU GENG WEI COMMODITY PURCHASE CO,YIWU GENG WEI COMMODITY PURCHASE,CO
1,1,4,SEW EURODRIVE GMBH & CO KG,S000,False,SEW EURODRIVE GMBH & CO KG,SEW EURODRIVE,GMBH & CO KG
2,2,49,PAM TRADING CORP,P500,False,PAM TRADING CORP,PAM TRADING,CORP
3,3,54,MORGAN AM&T SHANGHAI CO LTD,M625,False,MORGAN AM&T SHANGHAI CO LTD,MORGAN AM&T SHANGHAI,CO LTD
4,4,55,GROZ BECKERT KG,G620,False,GROZ BECKERT KG,GROZ BECKERT,KG


In [17]:
df['n_gram_fingerprint_n=2'] = df['No_Suffix'].apply(lambda x : fingerprinter(str(x).replace(' ','')).get_ngram_fingerprint(n=2))

In [19]:
df.head()

,Uniq_Index,index,Supp_Name,SOUNDEX_CODE,M/S,Supp_Name1,No_Suffix,suffix,n_gram_fingerprint_n=2
0,0,0,YIWU GENG WEI COMMODITY PURCHASE CO,Y000,False,YIWU GENG WEI COMMODITY PURCHASE CO,YIWU GENG WEI COMMODITY PURCHASE,CO,aschcodieiengegwhaicitiwmmmongodompurcsetyugur...
1,1,4,SEW EURODRIVE GMBH & CO KG,S000,False,SEW EURODRIVE GMBH & CO KG,SEW EURODRIVE,GMBH & CO KG,dreuewivodriroseurvewe
2,2,49,PAM TRADING CORP,P500,False,PAM TRADING CORP,PAM TRADING,CORP,adamdiinmtngparatr
3,3,54,MORGAN AM&T SHANGHAI CO LTD,M625,False,MORGAN AM&T SHANGHAI CO LTD,MORGAN AM&T SHANGHAI,CO LTD,aiamangaghhamomtnangorrgshts
4,4,55,GROZ BECKERT KG,G620,False,GROZ BECKERT KG,GROZ BECKERT,KG,beckecergrkeozrortzb


In [20]:
# df.to_excel('Applied_n=2_Unique_Supp_Name-20.xlsx',index=False)

In [24]:
df.drop_duplicates(subset=['No_Suffix'])

,Uniq_Index,index,Supp_Name,SOUNDEX_CODE,M/S,Supp_Name1,No_Suffix,suffix,n_gram_fingerprint_n=2
0,0,0,YIWU GENG WEI COMMODITY PURCHASE CO,Y000,False,YIWU GENG WEI COMMODITY PURCHASE CO,YIWU GENG WEI COMMODITY PURCHASE,CO,aschcodieiengegwhaicitiwmmmongodompurcsetyugur...
1,1,4,SEW EURODRIVE GMBH & CO KG,S000,False,SEW EURODRIVE GMBH & CO KG,SEW EURODRIVE,GMBH & CO KG,dreuewivodriroseurvewe
2,2,49,PAM TRADING CORP,P500,False,PAM TRADING CORP,PAM TRADING,CORP,adamdiinmtngparatr
3,3,54,MORGAN AM&T SHANGHAI CO LTD,M625,False,MORGAN AM&T SHANGHAI CO LTD,MORGAN AM&T SHANGHAI,CO LTD,aiamangaghhamomtnangorrgshts
4,4,55,GROZ BECKERT KG,G620,False,GROZ BECKERT KG,GROZ BECKERT,KG,beckecergrkeozrortzb
...,...,...,...,...,...,...,...,...,...
343326,343326,22450960,TAIZHOU HUANGYAN ZHENGGE MACHINERY,T200,False,TAIZHOU HUANGYAN ZHENGGE MACHINERY,TAIZHOU HUANGYAN ZHENGGE MACHINERY,,acaianchemenergegggyhehihohuinizmanengnzouryta...
343327,343327,22450988,HIROSE SEIKO TAI CHANG CO LTD,H620,False,HIROSE SEIKO TAI CHANG CO LTD,HIROSE SEIKO TAI CHANG,CO LTD,aiancheieshahiicikirkongosotroseta
343328,343328,22451283,KENJET TECHNOLGOY CO LTD,K523,False,KENJET TECHNOLGOY CO LTD,KENJET TECHNOLGOY,CO LTD,checenetgohnjekelgnjnooloytett
343331,343331,22452045,BPREX HEALTHCAREBROOKVILLE INC,B620,False,BPREX HEALTHCAREBROOKVILLE INC,BPREX HEALTHCAREBROOKVILLE,INC,alarbpbrcaeaebexhcheilkvlellltokooprrerothvixh
